# `Optimizable`. Documentation and examples.

Here we will step-by-step motivate the use of the `Optimizable` class.

In [1]:
from regelum import Optimizable
import torch as th
from torch.optim import Adam
import casadi as cs
import numpy as np
from regelum.optimizable.core.configs import torch_default_config, casadi_default_config
from regelum.__utilities import rc

## Torch optimization

Here is how one uses PyTorch to optimize a function in the simplest way:

1. Define the objective
2. Define the optimizer and pass decision variable (parameters to optimize) to it
3. zero_grad() -> evaluate objective -> backward() -> step() -> repeat

In [2]:
def objective_torch(x, y):
    """Whatever objective we have, we need to treat arguments as torch tensors."""
    return (x**2 + y**2).sum()


x = th.Tensor([1.0, 2.0]).requires_grad_(
    True
)  # Here we define a tensor containing parameters to optimize
opt_torch = Adam(params=[x], lr=0.1)

# The most frequent way to optimize some objective looks like this:
for _ in range(5000):
    opt_torch.zero_grad()
    obj = objective_torch(th.Tensor([1.0, 2.0]), x)
    obj.backward()
    opt_torch.step()

x  # Intended to be about zero for both elements of tensor.

tensor([-2.5223e-44,  0.0000e+00], requires_grad=True)

## Casadi optimization

Sometimes when we deal with nonlinear optimization we want to use CasADi, which is a python package
that offers an efficient and convenient toolset for symbolic optimization.

There are several APIs for nonlinear programming in CasADi. In this example we will use so-called `Opti` stack.

A birdview of a common optimization routine looks like this:

1. Create symbolic prototypes of variables and constant parameters
2. Make an inference of symbolic prototype for objective and constraints
3. Define an optimization problem and solve it with CasADi's optimization solver
4. Retrieve the solution

In [3]:
def objective_casadi(x, y):
    return cs.sumsqr(x) + cs.sumsqr(y)


opti = cs.Opti()  # Create an instance of Opti
x = opti.parameter(2)  # Create a constant parameter
y = opti.variable(2)  # Create a decistion variable

x, y

(MX(opti0_p_1), MX(opti0_x_1))

Let's observe the symbolic prototype of our objective.

In [4]:
objective_symb = objective_casadi(x, y)
objective_symb

MX((dot(opti0_p_1, opti0_p_1)+dot(opti0_x_1, opti0_x_1)))

In our current case we obtained a human readable expression (however it's not the case in general). Next, we just pass it into `.minimize()` method and define a solver.

After we call `.to_function()` method, we can invoke our optimization routine as a function that takes parameters as input arguments.

In [5]:
opti.minimize(objective_symb)
opti.solver(
    "ipopt",
    {"print_in": False, "print_out": False, "print_time": True},
    {"print_level": 0},
)
opt_func = opti.to_function(
    "min_fun", [x], [y, objective_symb], ["x"], ["y", "objective_casadi"]
)
opt_func(cs.DM([1.0, 2.0]))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  25.00us ( 25.00us)   1.54us (  1.54us)         1
  nlp_grad_f  | 105.00us ( 52.50us)   6.33us (  3.17us)         2
       total  |  10.99ms ( 10.99ms) 685.05us (685.05us)         1


(DM([0, 0]), DM(5))

## Regelum optimization

As we have seen, different frameworks can be used for optimization and actually they are pretty different in usage. In `Regelum` we want to interchange between optimization frameworks depending on context and purposes: sometimes we need to learn deep neural networks, sometimes we need to make constrained optimization that is easier to solve with CasADi.

This is where `Optimizable` comes in action.

In [6]:
def universal_objective(x, y):
    return rc.sum(x**2 + (y - 1) ** 2)

In [7]:
opt_regelum = Optimizable(casadi_default_config)
x = opt_regelum.create_variable(2, 1, name="x", is_constant=True)
y = opt_regelum.create_variable(2, 1, name="y")
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=cs.DM([1.0, 2.0]))

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  71.00us ( 35.50us)   4.16us (  2.08us)         2
  nlp_grad_f  | 111.00us ( 37.00us)   6.85us (  2.28us)         3
  nlp_hess_l  |  29.00us ( 29.00us)   1.76us (  1.76us)         1
       total  |  13.43ms ( 13.43ms) 839.06us (839.06us)         1


{'y': DM([1, 1])}

In [8]:
torch_default_config.config_options["n_epochs"] = 10000
opt_regelum = Optimizable(torch_default_config)
x = opt_regelum.create_variable(2, 1, name="x", is_constant=True)
y = opt_regelum.create_variable(
    2, 1, name="y", like=th.Tensor([1.0, 2.0]).requires_grad_(True)
)
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=th.FloatTensor([1.0, 2.0]))

VarContainer:
  y
  data: tensor([1.0000, 1.0000], requires_grad=True)
  metadata: tensor([1.0000, 1.0000], requires_grad=True)
  dims: (2, 1)
  is_constant: False


  

Working with models

In [9]:
from regelum.model import ModelWeightContainer, ModelWeightContainerTorch

In [10]:
model = ModelWeightContainer(2, cs.DM([[1.0, 2.0], [3.0, 4.0]]))
model("whatever")

DM([[1, 2]])

In [11]:
model_torch = ModelWeightContainerTorch(2)

In [12]:
model.weights, model.cache.weights

(DM(
 [[1, 2], 
  [3, 4]]),
 DM(
 [[1, 2], 
  [3, 4]]))

In [13]:
opt_regelum = Optimizable(casadi_default_config)
x = opt_regelum.create_variable(1, 2, name="x", is_constant=True)
w = opt_regelum.create_variable(name="w", like=model.named_parameters)
argin_to_model = opt_regelum.create_variable(
    1, 2, name="argi_to_model", is_constant=True
)
y = opt_regelum.create_variable(1, 2, name="y", is_nested_function=True)
opt_regelum.connect_source(connect_to=y, func=model, source=argin_to_model, weights=w)
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=cs.DM([1.0, 2.0]), y=cs.DM([1.0, 2.0]))

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   8.00us (  2.67us)   5.76us (  1.92us)         3
  nlp_grad_f  |   7.00us (  1.75us)   7.70us (  1.93us)         4
  nlp_hess_l  |   2.00us (  1.00us)   2.87us (  1.43us)         2
       total  | 810.00us (810.00us) 809.55us (809.55us)         1


{'w': DM(
 [[1, 1], 
  [0, 0]])}

In [14]:
opt_regelum = Optimizable(torch_default_config)
x = opt_regelum.create_variable(1, 2, name="x", is_constant=True)
w = opt_regelum.create_variable(name="w", like=model_torch.named_parameters)
argin_to_model = opt_regelum.create_variable(
    1, 2, name="argi_to_model", is_constant=True, like=th.FloatTensor([1.0, 2.0])
)
y = opt_regelum.create_variable(1, 2, name="y", is_nested_function=True)
opt_regelum.connect_source(
    connect_to=y, func=model_torch, source=argin_to_model, weights=w
)
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=th.FloatTensor([1.0, 2.0]))

VarContainer:
  w
  data: <bound method Module.named_parameters of ModelWeightContainerTorch()>
  metadata: <bound method Module.named_parameters of ModelWeightContainerTorch()>
  dims: ()
  is_constant: False


  y
  data: tensor([1.0000, 1.0000], grad_fn=<SliceBackward0>)
  metadata: tensor([1.0000, 1.0000], grad_fn=<SliceBackward0>)
  dims: (1, 2)
  is_constant: False


  

Inherit from `Optimizable`

In [15]:
from regelum.model import ModelPerceptron, ModelQuadLin

In [16]:
class MyOptimizableObject(Optimizable):
    def __init__(self, model, opt_config):
        super().__init__(opt_config)
        self.model = model
        self.initialize_optimization_procedure()

    def initialize_optimization_procedure(self):
        self.model_weights = self.create_variable(
            name="model_weights",
            like=self.model.named_parameters,
        )
        self.model_argin = self.create_variable(
            3, 2, name="model_argin", is_constant=True
        )
        self.model_output_reference = self.create_variable(
            3, 1, name="model_output_reference", is_constant=True
        )
        self.model_output = self.create_variable(
            name="model_output", is_nested_function=True
        )
        self.connect_source(
            connect_to=self.model_output,
            func=self.model,
            source=self.model_argin,
            weights=self.model_weights,
        )
        self.register_objective(
            self.objective_function, [self.model_output, self.model_output_reference]
        )
        self.register_constraint(self.constr, [self.model_output])

    def objective_function(self, model_output_reference, model_output):
        return rc.sum((model_output_reference - model_output) ** 2)

    def constr(self, model_output):
        return rc.max(4 - model_output)

In [17]:
torch_default_config.config_options["n_epochs"] = 50000
torch_default_config.config_options["is_reinstantiate_optimizer"] = True

model = ModelPerceptron(2, 1, 3, 3)
optbl = MyOptimizableObject(model, torch_default_config)

In [18]:
optbl.optimize(
    model_output_reference=th.FloatTensor([[1.0], [3], [5]]),
    model_argin=th.FloatTensor([[0.0, 0.0], [1, 2], [3, 5]]),
)

VarContainer:
  model_weights
  data: <bound method Module.named_parameters of ModelPerceptron(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layers): ModuleList(
    (0-2): 3 x Linear(in_features=3, out_features=3, bias=True)
  )
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)>
  metadata: <bound method Module.named_parameters of ModelPerceptron(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layers): ModuleList(
    (0-2): 3 x Linear(in_features=3, out_features=3, bias=True)
  )
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)>
  dims: ()
  is_constant: False


  model_output
  data: tensor([[6.7023],
        [6.7004],
        [6.7009]], grad_fn=<AddmmBackward0>)
  metadata: tensor([[6.7023],
        [6.7004],
        [6.7009]], grad_fn=<AddmmBackward0>)
  dims: ()
  is_constant: False


  

In [19]:
model = ModelQuadLin("symmetric", is_with_linear_terms=False, dim_inputs=2)

In [20]:
optbl = MyOptimizableObject(model, casadi_default_config)

In [21]:
new_weights = optbl.optimize(
    model_output_reference=cs.DM([[1.0], [3], [5]]),
    model_argin=cs.DM([[0.0, 0.0], [1, 2], [3, 5]]),
)["model_weights"]

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  18.00us (  1.50us)  16.49us (  1.37us)        12
       nlp_g  |  35.00us (  2.92us)  26.94us (  2.25us)        12
  nlp_grad_f  |  18.00us (  2.57us)  16.24us (  2.32us)         7
  nlp_hess_l  |  24.00us (  3.43us)  22.75us (  3.25us)         7
   nlp_jac_g  |  22.00us (  2.00us)  21.62us (  1.97us)        11
       total  |   2.55ms (  2.55ms)   2.54ms (  2.54ms)         1


In [22]:
new_weights

DM([-456.434, 492.245, -130.748])